In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import re
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ramaz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv('Restaurant_Reviews.csv', header=[0] , on_bad_lines='skip')
data.head()

,Review,Liked
0,Wow... Loved this place.,1.0
1,Crust is not good.,0.0
2,Not tasty and the texture was just nasty.,0.0
3,Stopped by during the late May bank holiday of...,1.0
4,The selection on the menu was great and so wer...,1.0


In [8]:
### PREPROCESSING ###
#####################
# Yorumların içindeki notlamaları ve sembolleri kaldırdık #
# [^a-zA-Z] küçük ve büyük harf dışındaki her şey demek ^ işareti olmazsa küçük ve büyük harf demek # 
from nltk.corpus import stopwords

yorumlar = []
for i in range(len(data)):
    yorum = re.sub('[^a-zA-Z]', ' ', data['Review'][i])
    yorum = yorum.lower()
    yorum = yorum.split()
    yorum = [ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('english'))]
    yorum = ' '.join(yorum)
    yorumlar.append(yorum)
yorumlar[0:3]

['wow love place', 'crust good', 'tasti textur nasti']

In [28]:
### FEATURE EXTRACTION ###
##########################
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(yorumlar).toarray()
y = data.Liked.fillna(0)
y.head()

0    1.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: Liked, dtype: float64

In [41]:
### MACHINE LEARNING ###
########################
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.7, random_state=0)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
preds = gnb.predict(X_valid)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_valid, preds)
print(f"Succes rate: {(cm[0][0] + cm[1][1]) / cm.sum() *100:.2f}%")

Succes rate: 68.37%
